In [ ]:
import healpy as hp

In [ ]:
ls *.pkl

In [ ]:
%matplotlib inline

In [ ]:
import pickle

In [ ]:
cl = {}
for i_tube in range(4):
    with open(f"N_ell_tube_ST{i_tube}.pkl", "rb") as f:
        cl[i_tube] = pickle.load(f)

In [ ]:
from so_models_v3 import SO_Noise_Calculator_Public_v3_1_1 as so_models

import matplotlib
matplotlib.rc('text', usetex=True)
fontProperties = {
                  'weight' : 'normal', 'size' : 16}
import matplotlib.pyplot as plt

####################################################################
####################################################################
##                   demonstration of the code
####################################################################

mode=1 # baseline
fsky=1
ellmax=1e4
el=None

dset_label = 'LAT\\_V3.1'
lat = so_models.SOSatV3point1(mode, el=el)
corr_pairs = [(0,1),(2,3),(4,5)]

print(dset_label)
bands = lat.get_bands()
print("band centers: ", lat.get_bands(), "[GHz]")
print("beam sizes: "  , lat.get_beams(), "[arcmin]")
N_bands = len(bands)

ell, N_ell_LA_T_full,N_ell_LA_P_full = lat.get_noise_curves(
    fsky, ellmax, 1, full_covar=True, deconv_beam=False, rolloff_ell=50)

WN_levels = lat.get_white_noise(fsky)**.5

N_ell_LA_T  = N_ell_LA_P_full[range(N_bands),range(N_bands)] / 2
N_ell_LA_P  = N_ell_LA_P_full[range(N_bands),range(N_bands)]

print("white noise levels: "  , WN_levels, "[uK-arcmin]")


In [ ]:
import matplotlib.pyplot as plt

In [ ]:
import mapsims

In [ ]:
from glob import glob

In [ ]:
import matplotlib
matplotlib.rc('text', usetex=False)
fontProperties = {'family':'sans-serif',
                  'weight' : 'normal', 'size' : 16}

In [ ]:
import pickle

In [ ]:
%matplotlib inline

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2,sharey=True, figsize=(20/3*2, 5*4))
pol = 0
i_tube = 0
for (tube, chs) in mapsims.so_utils.tubes.items():
    if tube.startswith("S"):
    
        m = {}
        for i, ch in enumerate(chs):
            len_cl = len(cl[i_tube][i])
            pixwin = hp.pixwin(8192 if len_cl>10000 else (len_cl)/3, lmax=len_cl-1)
            freq = mapsims.so_utils.parse_channels("SA_"+ch)[0].center_frequency.value
            ax[i_tube][i].loglog(ell, N_ell_LA_T[bands.searchsorted(freq)])
            ax[i_tube][i].loglog(cl[i_tube][i][pol])

            ax[i_tube][i].set_ylim(1e-8,1e1)
            ax[i_tube][i].set_title(tube + ": " + ch, fontsize=16)
            
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
        i_tube += 1
        
plt.savefig("SAT_T.png")

In [ ]:
fig, ax = plt.subplots(nrows=4, ncols=2, sharey=True, figsize=(20/3*2, 5*4))
ylim = (1e-6, 1e-1)
i_tube = 0
for (tube, chs) in mapsims.so_utils.tubes.items():
    if tube.startswith("S"):
    
        m = {}
        for i, ch in enumerate(chs):
            freq = mapsims.so_utils.parse_channels("SA_"+ch)[0].center_frequency.value
            print(ch, freq)
            ax[i_tube][i].loglog(ell, N_ell_LA_P[bands.searchsorted(freq)])
            ax[i_tube][i].set_ylabel(r"N_ell [muK^2]", fontsize=16)
        for pol in [1,2]:
            for i, ch in enumerate(chs):
                len_cl = len(cl[i_tube][i])
                pixwin = hp.pixwin(8192 if len_cl>10000 else (len_cl)/3, lmax=len_cl-1)
                ax[i_tube][i].loglog(cl[i_tube][i][pol])

                ax[i_tube][i].set_ylim(*ylim)
                ax[i_tube][i].set_title(tube + ": " + ch, fontsize=16)
        i_tube += 1
    

plt.savefig("SAT_P.png")